In [66]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [100]:
import pandas as pd #Basic Preprocessing
import numpy as np #Basic Preprocessing
import seaborn as sns #Data Visualization
import matplotlib.pyplot as plt #Data Visualization
from sklearn.metrics import classification_report #Evaluation Metrics
from sklearn.metrics import confusion_matrix #Confusion Matrix
from sklearn.model_selection import train_test_split #Test Train Split
from sklearn.preprocessing import LabelEncoder #For Label Encoding
from sklearn.metrics import roc_auc_score, roc_curve, auc #For ROC, AUC

In [68]:
train = pd.read_csv('../input/unimich-ds-assignment4/train.csv', encoding='ISO-8859-1', dtype={'zip_code': str, 'non_us_str_code': str, 'grafitti_status': str, 'violator_name':str, 
                            'mailing_address_str_number': str}) #Reading the training the Dataset
train.set_index(train['ticket_id'],inplace=True) #Setting the Primary Key as the Index Value

In [69]:
train.shape #Gives the rows and columns

In [70]:
train.info() #To check the Data Type

In [71]:
train.describe() #Basic Descriptive Stats

In [72]:
#This Function finds the unique values in every column of the dataframe and returns a dictonary.
def unique_values(train):
    d={}
    for i in train.columns:
        d[i]=len(train[i].unique())
    return d

In [73]:
unique_values(train)

In [74]:
train.corr()

In [75]:
#Finding the classes count in the target variable
train["compliance"].value_counts()

#There is a class imbalance, but considering this is a real world dataset, it can be attributed to a real world scenario.
#Methods to solve include, making the classes equal by reducing the 0' s or augmenting the data.

In [76]:
#Keeping Columns based on the observed Corelation and Unique Values count
feature_names=['disposition','violation_code', "late_fee", "discount_amount", "compliance"]
train=train[feature_names]

In [77]:
#To check the Null Values
train.isna().sum()

In [78]:
train.dropna(subset=['compliance'],inplace=True) #Dropping the Null Values
train.fillna(method='pad', inplace=True) #Filling in the Null Values

In [79]:
#Label Encode the Categorical Data Values
label_encoder = LabelEncoder()
train['disposition']=label_encoder.fit_transform(train['disposition'])


label_encoder = LabelEncoder()
train['violation_code'] = label_encoder.fit_transform(train['violation_code'])

In [80]:
train.info()

In [81]:
sns.boxplot(x=train["discount_amount"]) #Checking the Distribution using a box plot

In [82]:
sns.boxplot(x=train["late_fee"])

In [ ]:
#Suppose if Outliers need to be removed
#from sklearn.neighbors import LocalOutlierFactor
# lof = LocalOutlierFactor()
# yhat = lof.fit_predict(X_train.values)
# yhat
# # select all rows that are not outliers
# mask = yhat != -1
# print(mask)
# X_train, y_train = X_train.values[mask, :], y_train.values[mask]

In [83]:
sns.heatmap(train.corr())
#Disount Amount > Disposition> Late Fee > Violation Code

In [88]:
#Splitting the Dataset in the ratio of 80:20
#Random set is set to 42
y = train['compliance']

X=train[['disposition','violation_code', "late_fee", "discount_amount"]]
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42) 

Model 1- **Random Forest**

In [96]:
def randomForestRegressor():
    from sklearn.ensemble import RandomForestRegressor
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    fpr_lr, tpr_lr, _ = roc_curve(y_test, model.predict(X_test))
    roc_auc_lr = auc(fpr_lr, tpr_lr)
    y_predict= pd.DataFrame(model.predict(X_test),index=X_test.index,columns=['compliance'])
    plt.plot(fpr_lr, tpr_lr)
    plt.show()
    
    return roc_auc_lr
randomForestRegressor()

In [101]:
def randomForestClassifier():
    from sklearn.ensemble import RandomForestClassifier
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    fpr_lr, tpr_lr, _ = roc_curve(y_test, model.predict_proba(X_test)[:,1])
    roc_auc_lr = auc(fpr_lr, tpr_lr)
    y_predict= pd.DataFrame(model.predict(X_test),index=X_test.index,columns=['compliance'])
    plt.plot(fpr_lr, tpr_lr)
    plt.show()
    
    print("Classification Report: ")
    print(classification_report(y_test,model.predict(X_test)))
    print("Confuion Matrix")
    print(confusion_matrix(y_test,model.predict(X_test)))
    return roc_auc_lr
randomForestClassifier()

Model 2- **XGBoost**

In [104]:
def XGBoostRegressor():
    import xgboost as xg
    model = xg.XGBRegressor(objective ='reg:linear',
                  n_estimators = 100, seed = 123,max_depth = 5)
    model.fit(X_train, y_train)
    fpr_lr, tpr_lr, _ = roc_curve(y_test, model.predict(X_test))
    roc_auc_lr = auc(fpr_lr, tpr_lr)
    y_predict= pd.DataFrame(model.predict(X_test),index=X_test.index,columns=['compliance'])
    plt.plot(fpr_lr, tpr_lr)
    plt.show()
    
    return roc_auc_lr
XGBoostRegressor()

In [107]:
def XGClassifier():
    import xgboost as xg
    model = xg.XGBClassifier( n_estimators = 100, seed = 123,max_depth = 5)
                
    model.fit(X_train, y_train)
    fpr_lr, tpr_lr, _ = roc_curve(y_test, model.predict_proba(X_test)[:,1])
    roc_auc_lr = auc(fpr_lr, tpr_lr)
    y_predict= pd.DataFrame(model.predict(X_test),index=X_test.index,columns=['compliance'])
    plt.plot(fpr_lr, tpr_lr)
    plt.show()
    
    print("Classification Report: ")
    print(classification_report(y_test,model.predict(X_test)))
    print("Confuion Matrix")
    print(confusion_matrix(y_test,model.predict(X_test)))
    return roc_auc_lr
XGClassifier()

Model 3- **Gradient Boosted Decision Tree**

In [112]:
def GradientBoostedDecisonTreeRegressor():
    from sklearn.ensemble import GradientBoostingRegressor
    model = GradientBoostingRegressor(
    max_depth=2,
    n_estimators=3,
    learning_rate=1.0
)
    model.fit(X_train, y_train)
    fpr_lr, tpr_lr, _ = roc_curve(y_test, model.predict(X_test))
    roc_auc_lr = auc(fpr_lr, tpr_lr)
    plt.plot(fpr_lr, tpr_lr)
    plt.show()

    y_predict= pd.DataFrame(model.predict(X_test),index=X_test.index,columns=['compliance'])
    return roc_auc_lr
GradientBoostedDecisonTreeRegressor()

In [126]:
def GradientBoostingClassifier():
  from sklearn.ensemble import GradientBoostingClassifier
  model = GradientBoostingClassifier(
    max_depth=2,
    n_estimators=3,
    learning_rate=1.0
)
  model.fit(X_train,y_train)
  fpr_lr, tpr_lr, _ = roc_curve(y_test, model.predict_proba(X_test)[:,1])
  roc_auc_lr = auc(fpr_lr, tpr_lr)
  y_predict= pd.DataFrame(model.predict(X_test),index=X_test.index,columns=['compliance'])
  plt.plot(fpr_lr, tpr_lr)
  plt.show()
  
  print("Classification Report: ")
  print(classification_report(y_test,model.predict(X_test)))
  print("Confuion Matrix")
  print(confusion_matrix(y_test,model.predict(X_test)))
  return roc_auc_lr
GradientBoostingClassifier()

Model 4- **KNN**

In [125]:
def KnnRegressor():
    from sklearn.neighbors import KNeighborsRegressor
    d={}
    for i in range(5, 15): 
      model = KNeighborsRegressor(n_neighbors = i)
      model.fit(X_train, y_train)
      fpr_lr, tpr_lr, _ = roc_curve(y_test, model.predict(X_test))
      roc_auc_lr = auc(fpr_lr, tpr_lr)
      d[i]=roc_auc_lr
      y_predict= pd.DataFrame(model.predict(X_test),index=X_test.index,columns=['compliance'])
    new_val = d.values()
    maximum_val = max(new_val)
    
    return d, maximum_val
KnnRegressor()

In [ ]:
def KnnClassifer():
    from sklearn.neighbors import KNeighborsClassifier
    d={}
    for i in range(5, 15): 
      model = KNeighborsClassifier(n_neighbors = i)
      model.fit(X_train, y_train)
      fpr_lr, tpr_lr, _ = roc_curve(y_test, model.predict_proba(X_test)[:,1])
      roc_auc_lr = auc(fpr_lr, tpr_lr)
      d[i]=roc_auc_lr
      y_predict= pd.DataFrame(model.predict(X_test),index=X_test.index,columns=['compliance'])
    new_val = d.values()
    maximum_val = max(new_val)
    
    return d, maximum_val
KnnClassifer()

Model 5- **MultiLayerPerceptron**

In [127]:
def MLPRegressor():
    from sklearn.neural_network import MLPRegressor
    model = MLPRegressor(solver='lbfgs', activation = 'tanh',
                         alpha = 1.0,
                         hidden_layer_sizes = [100, 100],
                         random_state = 0)
    model.fit(X_train, y_train)
    fpr_lr, tpr_lr, _ = roc_curve(y_test, model.predict(X_test))
    roc_auc_lr = auc(fpr_lr, tpr_lr)
    y_predict= pd.DataFrame(model.predict(X_test),index=X_test.index,columns=['compliance'])
    
    plt.plot(fpr_lr, tpr_lr)
    plt.show()
    
    return roc_auc_lr
MLPRegressor()

In [ ]:
def MLPClassifier():
    from sklearn.neural_network import MLPClassifier
    model = MLPClassifier(solver='lbfgs', activation = 'tanh',
                         alpha = 1.0,
                         hidden_layer_sizes = [100, 100],
                         random_state = 0)
    model.fit(X_train, y_train)
    fpr_lr, tpr_lr, _ = roc_curve(y_test, model.predict_proba(X_test)[:,1])
    roc_auc_lr = auc(fpr_lr, tpr_lr)
    y_predict= pd.DataFrame(model.predict(X_test),index=X_test.index,columns=['compliance'])
    
    plt.plot(fpr_lr, tpr_lr)
    plt.show()
    
    print("Classification Report: ")
    print(classification_report(y_test,model.predict(X_test)))
    print("Confuion Matrix")
    print(confusion_matrix(y_test,model.predict(X_test)))
    
    return roc_auc_lr
MLPClassifier()

Model 6- **Stacking**

In [ ]:
def StackingRegressor():
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.ensemble import StackingRegressor
    from sklearn.neural_network import MLPRegressor
    import xgboost as xg

    model1 = RandomForestRegressor()
    model1.fit(X_train, y_train)
    
    model2 = MLPRegressor()
    model2.fit(X_train, y_train)

    model3 = xg.XGBRegressor()
    model3.fit(X_train, y_train)
    
    
    estimators = [('rf' , RandomForestRegressor()),('mlp',MLPRegressor()),('xgb',xg.XGBRegressor()) ]

    reg = StackingRegressor(estimators=estimators,final_estimator=RandomForestRegressor(n_estimators=10,
                                              random_state=42))
    
    reg.fit(X_train, y_train)
    fpr_lr, tpr_lr, _ = roc_curve(y_test, reg.predict(X_test))
    roc_auc_lr = auc(fpr_lr, tpr_lr)
    y_predict= pd.DataFrame(reg.predict(X_test),index=X_test.index,columns=['compliance'])
    
    plt.plot(fpr_lr, tpr_lr)
    plt.show()
    
    
    return roc_auc_lr
StackingRegressor()

In [ ]:
def StackingClassifier():
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.ensemble import StackingClassifier
    from sklearn.neural_network import MLPClassifier
    import xgboost as xg

    model1 = RandomForestClassifier()
    model1.fit(X_train, y_train)
    
    model2 = MLPClassifier()
    model2.fit(X_train, y_train)

    model3 = xg.XGBClassifier()
    model3.fit(X_train, y_train)
    
    estimators = [('rf' , RandomForestClassifier()),('mlp',MLPClassifier()),('xgb',xg.XGBClassifier()) ]

    reg = StackingClassifier(estimators=estimators,final_estimator=RandomForestClassifier(n_estimators=10,
                                              random_state=42))
    
    reg.fit(X_train, y_train)
    fpr_lr, tpr_lr, _ = roc_curve(y_test, reg.predict_proba(X_test)[:,1])
    roc_auc_lr = auc(fpr_lr, tpr_lr)
    y_predict= pd.DataFrame(reg.predict(X_test),index=X_test.index,columns=['compliance'])
    
    plt.plot(fpr_lr, tpr_lr)
    plt.show()
    
    print("Classification Report: ")
    print(classification_report(y_test,reg.predict(X_test)))
    print("Confuion Matrix")
    print(confusion_matrix(y_test,reg.predict(X_test)))
    return roc_auc_lr
StackingClassifier()

Model 7- **DecisionTree**

In [ ]:
def DecisionTreeRegressor():
    from sklearn.tree import DecisionTreeRegressor
    model =DecisionTreeRegressor(max_depth = 10)
    model.fit(X_train, y_train)
    fpr_lr, tpr_lr, _ = roc_curve(y_test, model.predict(X_test))
    roc_auc_lr = auc(fpr_lr, tpr_lr)
    y_predict= pd.DataFrame(model.predict(X_test),index=X_test.index,columns=['compliance'])
    
    plt.plot(fpr_lr, tpr_lr)
    plt.show()
    
    return roc_auc_lr
DecisionTreeRegressor()

In [ ]:
def DecisionTreeClassifer():
    from sklearn.tree import DecisionTreeClassifier
    model =DecisionTreeClassifier(max_depth = 10)
    model.fit(X_train, y_train)
    fpr_lr, tpr_lr, _ = roc_curve(y_test, model.predict_proba(X_test)[:,1])
    roc_auc_lr = auc(fpr_lr, tpr_lr)
    y_predict= pd.DataFrame(model.predict(X_test),index=X_test.index,columns=['compliance'])
    
    plt.plot(fpr_lr, tpr_lr)
    plt.show()
    
    
    print("Classification Report: ")
    print(classification_report(y_test,model.predict(X_test)))
    print("Confuion Matrix")
    print(confusion_matrix(y_test,model.predict(X_test)))
    
    return roc_auc_lr
DecisionTreeClassifer()

Difference between Linear and Logistic Regression

In [129]:
def LinearRegression():
    from sklearn.linear_model import LinearRegression
    model = LinearRegression()
    model.fit(X_train, y_train)
    fpr_lr, tpr_lr, _ = roc_curve(y_test, model.predict(X_test))
    roc_auc_lr = auc(fpr_lr, tpr_lr)
    y_predict= pd.DataFrame(model.predict(X_test),index=X_test.index,columns=['compliance'])
    
    plt.plot(fpr_lr, tpr_lr)
    plt.show()
    
    return roc_auc_lr
LinearRegression()

In [130]:
def LogisticRegressor():
  from sklearn.linear_model import LogisticRegression

  model = LogisticRegression()
  model.fit(X_train, y_train)
  fpr_lr, tpr_lr, _ = roc_curve(y_test, model.predict_proba(X_test)[:,1])
  roc_auc_lr = auc(fpr_lr, tpr_lr)
  
  plt.plot(fpr_lr, tpr_lr)
  plt.show()
    
  print("Classification Report: ")
  print(classification_report(y_test,model.predict(X_test)))
  print("Confuion Matrix")
  print(confusion_matrix(y_test,model.predict(X_test)))
  y_predict= pd.DataFrame(model.predict(X_test),index=X_test.index,columns=['compliance'])
  
  return roc_auc_lr
LogisticRegressor()

In [137]:
mydata = [
    ["Linear Regression", LinearRegression()],
    ["Logistic Regression",LogisticRegressor()],
    ["Decsion Tree Regression",DecisionTreeRegressor()],
    ["Decision Tree Classifier", DecisionTreeClassifier()],
    ["Random Forest Regressor",randomForestRegressor() ],
    ["Random Forest Classifier",randomForestClassifier() ],
    ["Stacking Regressor", StackingRegressor()],
    ["Stacking Classifier",StackingClassifier() ],
    ["XGBosst Regressor", XGBoostRegressor()],
    ["XGBosst Classifier",XGClassifier()],
    ["KNN Regressor", KnnRegressor()],
    ["KNN Classifier", KnnClassifier()],
    ["Gradient Boosting DecisionTree Regressor",GradientBoostedDecisonTreeRegressor() ],
    ["Gradient Boosting DecisionTree Classifier",GradientBoostedClassifier()],
    ["MLP Regressor", MLPRegressor()],
    ["MLP Classifier",MLPClassifier()],
]

In [139]:
from tabulate import tabulate
head=["ModelName", "AUC Score"]
print(tabulate(mydata, headers=head, tablefmt="grid"))